In [1]:
from src.data import _load_readmission_dataset
from src.helper_functions import _prepare_rt_tensors
from src.models import *
from src.losses import *
from src.training import *
from src.metrics import *

In [2]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
x, t, e, d = _load_readmission_dataset(sequential=True)

/home/ayush/DeepReEvent/src/data.py:47: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data['max_time'] = data.groupby('id')['t.stop'].transform(max)


In [4]:
readmission_tensor = _prepare_rt_tensors(x, t, e, d)
print(readmission_tensor.keys())
locals().update(readmission_tensor) # create variables from dictionary

dict_keys(['x_train', 'x_val', 'x_test', 't_train', 't_val', 't_test', 'e_train', 'e_val', 'e_test', 'd_train', 'd_val', 'd_test'])


/home/ayush/DeepReEvent/src/helper_functions.py:83: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  x = torch.tensor(x, dtype=torch.float32)


In [5]:
# model and training parameters

input_size = len(x[0]) 
output_size = int(max(t))
hidden_size = 16   # Number of units in the RNN layer

model = SimpleRNN(input_size, hidden_size, output_size, 2)

# Instantiate the model

num_epochs = 200
patience = 3  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
wait = 0
loss_function = recurrent_terminal_loss

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-7)

In [6]:
#prepare data loaders

train_dataset = torch.utils.data.TensorDataset(x_train, t_train, e_train, d_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)

val_dataset = torch.utils.data.TensorDataset(x_val, t_val, e_val, d_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

In [7]:
train_validate_rt_model(model, train_dataloader, val_dataloader, loss_function, optimizer, num_epochs, patience, print_every=10)

Epoch: 0   Training loss: 4.3483   Validation loss: 3.7823


Epoch: 10   Training loss: 2.3501   Validation loss: 2.0602
Epoch: 20   Training loss: 2.0972   Validation loss: 1.8481
Epoch: 30   Training loss: 2.0427   Validation loss: 1.7889
Epoch: 40   Training loss: 2.0135   Validation loss: 1.7527
Epoch: 50   Training loss: 1.9776   Validation loss: 1.6760
Epoch: 60   Training loss: 1.9019   Validation loss: 1.5589
Early stopping


In [8]:
test_predictions = model(x_test).squeeze(-1)

survival_predictions = test_predictions[:, :, 0:1].squeeze(-1)
recurrent_predictions = test_predictions[:, :, 1:2].squeeze(-1)

In [9]:
calculate_survival_metrics(survival_predictions, d_train, t_train, d_test, t_test, d_val, t_val)

{'Brier Score': array([0.03264354, 0.02863364, 0.02086965]),
 '25th Quantile CI': 0.9838266742464028,
 '50th Quantile CI': 0.9838266742464028,
 '75th Quantile CI': 0.9838266742464028}